In [0]:
# google driveのファイルを使う時に使用
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# google check auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# google driveのファイル取得
downloaded = drive.CreateFile({'id':'1OC4Ecen82Qakq1VhrzWOaVT_goqgniKf'})

# Download the file to a local disk as 'sample.csv'.
downloaded.GetContentFile('ml-100k.zip')

In [3]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [0]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from scipy import sparse

In [5]:
df = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id','item_id', 'rating', 'timestamp'])
df.max()

user_id            943
item_id           1682
rating               5
timestamp    893286638
dtype: int64

In [0]:
shape = (df.max().loc['user_id'], df.max().loc['item_id'])
R = np.zeros(shape) 

for i in df.index:
    row = df.loc[i]
    R[row['user_id'] -1 , row['item_id'] - 1] = row['rating']

In [60]:
print(R.todense())

AttributeError: ignored

In [0]:
def compute_item_similarities(R):
    # n: movie counts
    n = R.shape[1]
    sims = np.zeros((n,n))

    for i in range(n):
        for j in range(i, n):
            if i == j:
                sim = 1.0
            else:
                # R[:, i] は アイテム i に関する全ユーザの評価を並べた列ベクトル
                sim = similarity(R[:,i], R[:,j])

            sims[i][j] = sim 
            sims[j][i] = sim 

    return sims 

In [0]:
def similarity(item1, item2):
    # item1 と item2 のどちらも評価済であるユーザの集合
    common = np.logical_and(item1 != 0, item2 != 0)

    v1 = item1[common]
    v2 = item2[common]

    sim = 0.0
    # 共通評価者が 2以上という制約にしている
    if v1.size > 1:
        sim = 1.0 - distance.cosine(v1, v2)

    return sim

In [69]:
sims = compute_item_similarities(R)
sims

array([[1.        , 0.94873739, 0.91329972, ..., 0.        , 0.        ,
        0.        ],
       [0.94873739, 1.        , 0.90887971, ..., 0.        , 0.        ,
        0.        ],
       [0.91329972, 0.90887971, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [0]:
def predict(u, sims):
  x = np.zeros(u.size)
  x[u > 0] = 1

  scores = sims.dot(u)
  normalizers = sims.dot(x)

  prediction = np.zeros(u.size)
  for i in range(u.size):
     if normalizers[i] == 0 or u[i] > 0:
        prediction[i] = 0
     else:
      prediction[i] = scores[i] / normalizers[i]

  return prediction

In [83]:
u = np.array([5, 0, 1])
sims = np.array([ [1, 0.2, 0], [0.2, 1, 0.1], [0, 0.1, 1] ])

print(predict(u,sims))

[0.         3.66666667 0.        ]
